In [2]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import math
from torch_scatter import scatter_add
import torch_sparse
class Gtconv(nn.Module):
    def __init__(self,in_channels,out_chanels,num_nodes):

        super(Gtconv,self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.weight = nn.Parameter(torch.Tensor(out_channels,in_channels))
        self.bias = None
        self.num_nodes = num_nodes
        self.reset_parameters()
    def reset_parameters(self):
        n = self.in_channels
        nn.init.normal_(self.weight, std=0.01)
        if self.bias is not None:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            nn.init.uniform_(self.bias, -bound, bound)
    def forward(self,A,num_nodes):
        filter=F.softmax(self.weight,dim=1)
        num_channels=filter.shape[0]
        result=[]
        for i in range (num_channels):
            for j,(edge_index,value) in enumerate(A):
                if j==0:
                    combined_edge_index=edge_index
                    combined_edge_value=value*filter[i][j]
                else:
                    combined_edge_index=torch.cat((combined_edge_index,edge_index),dim=1)
                    combined_edge_value=torch.cat((combined_edge_value,value*filter[i][j]))
            index,value=torch_sparse.coalesce(combined_edge_index.detach(),combined_edge_value,m=num_nodes,n=num_nodes,op='add')
            result.append(index,value)
        return result
    

            


ModuleNotFoundError: No module named 'torch_sparse'

In [2]:
import torch

print("PyTorch Version:", torch.__version__)
print("CUDA Version:", torch.version.cuda)


PyTorch Version: 1.13.1+cu117
CUDA Version: 11.7


In [ ]:
pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.13.1+cu117.html


Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1+cu117.html
  Using cached torch_scatter-2.1.2.tar.gz (108 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=3645411 sha256=1d9b462720b19b35210ac981212692b217a2a198379dd842b6980d86e45c640b
  Stored in directory: /home/accl-ros/.cache/pip/wheels/b0/1b/e7/71279bdf40e70266e71844d4c25ae7dd0dc9cc2d8683acd9b2
Successfully built torch-scatter
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
if 'IS_GRADESCOPE_ENV' not in os.environ:
  torch_version = str(torch.__version__)
  scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  !pip install torch-scatter -f $scatter_src
  !pip install torch-sparse -f $sparse_src
  !pip install torch-geometric
  !pip install ogb

Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1+cu117.html
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1+cu117.html
  Using cached torch_sparse-0.6.18.tar.gz (209 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp310-cp310-linux_x86_64.whl size=2625149 sha256=65a27795f6b456832b473996b3c16eb5ff0636dccc65ee6c296a56b49715461a
  Stored in directory: /home/accl-ros/.cache/pip/wheels/fe/a5/2a/06c54109734da4360822f54cd576d6b5c7f71e378d3e5110c8
Successfully built torch-sparse


In [ ]:
filter=torch.Tensor(2,2)
filter[0][1]

tensor(2.0228e+20)

In [ ]:
class GTLayer(nn.Module):
    def __init__(self,in_channels,out_channels,num_nodes,first=True):
        super(GTLayer,self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.first = first
        self.num_nodes = num_nodes
        if self.first==True:
            self.conv1=Gtconv(in_channels,out_channels,num_nodes)
            self.conv2=Gtconv(in_channels,out_channels,num_nodes)
        else:
            self.conv1=Gtconv(in_channels,out_channels,num_nodes)
        def forward(self,A,num_nodes,H=None):
            if self.first==True:
                q1=self.conv1(A,num_nodes)
                q2=self.conv2(A,num_nodes)
                W=[(F.softmax(self.conv1.weight,dim=1)),(F.softmax(self.conv2.weight,dim=1))]
            else:
                q2=self.conv1=Gtconv(in_channels,out_channels,num_nodes)
                Q1=H
                W=[(F.softmax(self.conv1.weight,dim=1))]
            H=[]

            for i in range(len(q1)):  
                q1_edge,q1_value=q1[i]
                q2_edge,q2_value=q2[i]
                mat_q1=torch.sparse_coo_tensor(q1_edge,q1_value,(num_nodes,num_nodes))
                mat_q2=torch.sparse_coo_tensor(q2_edge,q2_value,(num_nodes,num_nodes))
                mat=torch.sparse.mm(mat_q1,mat_q2).coalesce()
                edges,values=mat.indices(),mat.values()
                H.append((edges,values))
            return H,W






In [5]:
class GTN(nn.Module):
    
    def __init__(self, num_edge, num_channels, w_in, w_out, num_class, num_nodes, num_layers, args=None):
        super(GTN, self).__init__()
        self.num_edge = num_edge
        self.num_channels = num_channels
        self.num_nodes = num_nodes
        self.w_in = w_in
        self.w_out = w_out
        self.num_class = num_class
        self.num_layers = num_layers
        self.args = args
        layers = []
        for i in range(num_layers):
            if i==0:
                layers.append(GTLayer(num_edge,num_channels,num_nodes,first=True))
            else:
                layers.append(GTLayer(num_edge, num_channels, num_nodes, first=False))
        self.layers=nn.ModuleList(layers)
        self.loss=nn.CrossEntropyLoss()
        self.gcn = Gtconv(in_channels=self.w_in, out_channels=w_out, args=args)
        self.linear = nn.Linear(self.w_out*self.num_channels, self.num_class)
    def normalization(self, H, num_nodes):
        norm_H = []
        for i in range(self.num_channels):
            edge, value=H[i]
            deg_row, deg_col = self.norm(edge.detach(), num_nodes, value)
            value = (deg_row) * value
            norm_H.append((edge, value))
        return norm_H

    def norm(self, edge_index, num_nodes, edge_weight, improved=False, dtype=None):
        if edge_weight is None:
            edge_weight = torch.ones((edge_index.size(1), ),
                                    dtype=dtype,
                                    device=edge_index.device)
        edge_weight = edge_weight.view(-1)
        assert edge_weight.size(0) == edge_index.size(1)
        row, col = edge_index
        deg = scatter_add(edge_weight.clone(), row, dim=0, dim_size=num_nodes)
        deg_inv_sqrt = deg.pow(-1)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0

        return deg_inv_sqrt[row], deg_inv_sqrt[col]
    def forward(self, A, X, target_x, target, num_nodes=None, eval=False, node_labels=None):
        if num_nodes is None:
            num_nodes = self.num_nodes
        Ws = []
        for i in range(self.num_layers):
            if i==0:
                H,W=self.layers[i](A,num_nodes,eval=eval)
            else:
                H,W=self.layers[i](A,num_nodes,H,evel=eval)
            H=self.normalization(H,num_nodes)
            Ws.append(W)
        for i in range (self.num_channels):
            edge_index,edge_weight=H[i][0],H[i][1]
            if i==0:
                x=self.gcn(X,edge_index,edge_weight)
            else:
                x_tmp=x=self.gcn(X,edge_index,edge_weight)
                X=torch.cat((X,x_tmp),dim=1)
        y=self.linear(X[target_x])
        if eval:
            return y
        else:
            loss=self.loss(y,target)
        return loss,y,Ws
            

    

In [6]:
from scipy import io
import numpy as np
from scipy.sparse import csr_matrix
mat_file = io.loadmat('ACM.mat')



In [7]:
mat_file

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Mon Aug 08 18:23:50 2011',
 '__version__': '1.0',
 '__globals__': [],
 'TvsP': <1903x12499 sparse matrix of type '<class 'numpy.float64'>'
 	with 972973 stored elements in Compressed Sparse Column format>,
 'PvsA': <12499x17431 sparse matrix of type '<class 'numpy.float64'>'
 	with 37055 stored elements in Compressed Sparse Column format>,
 'PvsV': <12499x196 sparse matrix of type '<class 'numpy.float64'>'
 	with 12499 stored elements in Compressed Sparse Column format>,
 'AvsF': <17431x1804 sparse matrix of type '<class 'numpy.float64'>'
 	with 30424 stored elements in Compressed Sparse Column format>,
 'VvsC': <196x14 sparse matrix of type '<class 'numpy.float64'>'
 	with 196 stored elements in Compressed Sparse Column format>,
 'PvsL': <12499x73 sparse matrix of type '<class 'numpy.float64'>'
 	with 12499 stored elements in Compressed Sparse Column format>,
 'PvsC': <12499x14 sparse matrix of type '<class 'numpy.fl

In [12]:
paper_conference=mat_file['PvsC']
paper_conference=paper_conference.nonzero()[1]

In [16]:
import numpy as np
paper_db=np.isin(paper_conference,[1,13])

In [19]:
paper_db_idx=np.where(paper_db==True)[0]

In [23]:
paper_db_idx=np.sort(np.random.choice(paper_db_idx,994,replace=False))

In [26]:
paper_dm = np.isin(paper_conference,[0])
paper_dm_idx = np.where(paper_dm == True)[0]

paper_wc = np.isin(paper_conference,[9,10])
paper_wc_idx = np.where(paper_wc == True)[0]

In [28]:
paper_idx=np.sort(list(paper_db_idx)+list(paper_dm_idx)+list(paper_wc_idx))


In [29]:
len(paper_idx)

3025

In [34]:
paper_target=[]
for idx in paper_idx:
    if idx in paper_dm_idx:
        paper_target.append(0)
    elif idx in paper_wc_idx:
        paper_target.append(1)
    else:
        paper_target.append(2)
paper_target=np.array(paper_target)

In [56]:
paper_target.shape

(3025,)

In [53]:
authors=mat_file['PvsA'][paper_idx].nonzero()[1]
author_dic={}
reidx_author=[]
for author in authors:
    if author not in author_dic:
        author_dic[author]=len(paper_idx)+len(author_dic)
    reidx_author.append(author_dic[author])

reidx_author=np.array(reidx_author)

In [54]:
subjects = mat_file['PvsL'][paper_idx].nonzero()[1]
subject_dic = {}
re_subjects = []
for subject in subjects:
    if subject not in subject_dic:
        subject_dic[subject] = len(subject_dic) + len(paper_idx) + len(author_dic)
    re_subjects.append(subject_dic[subject])
re_subjects = np.array(re_subjects)

In [55]:
node_num = len(paper_idx) + len(author_dic) + len(subject_dic)
node_num

9060

In [60]:
papers=mat_file['PvsA'][paper_idx].nonzero()[0]
np.ones_like(papers)

array([1, 1, 1, ..., 1, 1, 1], dtype=int32)

In [32]:
import torch
from torch_scatter import scatter_add
from torch_sparse import coalesce
import torch.nn.functional as F
import torch.nn as nn
import math

# Define the GTConv class
class GTConv(nn.Module):
    
    def __init__(self, in_channels, out_channels, num_nodes):
        super(GTConv, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.weight = nn.Parameter(torch.Tensor(out_channels, in_channels))
        self.bias = None
        self.num_nodes = num_nodes
        self.reset_parameters()
        
    def reset_parameters(self):
        nn.init.normal_(self.weight, std=0.01)
        if self.bias is not None:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            nn.init.uniform_(self.bias, -bound, bound)

    def forward(self, A, num_nodes, eval=eval):
        filter = F.softmax(self.weight, dim=1)
        
        num_channels = filter.shape[0]
        
        results = []
        for i in range(num_channels):
            print(f"Processing output channel {i}")
            for j, (edge_index, edge_value) in enumerate(A):
                print(f"    Processing edge type {j}")
                print(edge_index)
                print(filter[i][j])
                if j == 0:
                    total_edge_index = edge_index
                    total_edge_value = edge_value * filter[i][j]
                else:
                    total_edge_index = torch.cat((total_edge_index, edge_index), dim=1)
                    print('inner',total_edge_index)
                    total_edge_value = torch.cat((total_edge_value, edge_value * filter[i][j]))
            print('after j',total_edge_index)
            print(total_edge_value)
            
            index, value = coalesce(total_edge_index.detach(), total_edge_value, m=num_nodes, n=num_nodes, op='add')
            print('index',index)
            print('valu',value)
            results.append((index, value))
        return results

# Sample input A
# Define a new A with adjusted edge_value shapes



# Number of nodes
num_nodes = 5

# Instantiate GTConv
gtconv = GTConv(in_channels=3, out_channels=2, num_nodes=num_nodes)

# Perform forward pass
output_results = gtconv(A, num_nodes)

# Print the results
print("\nFinal Results:")
for i, result in enumerate(output_results):
    print(f"Output Channel {i}: {result}")


Processing output channel 0
    Processing edge type 0
tensor([[1, 0, 2, 1],
        [0, 1, 1, 2]], device='cuda:0')
tensor(0.3314, grad_fn=<SelectBackward0>)
    Processing edge type 1
tensor([[1, 2, 0, 0],
        [0, 0, 1, 2]], device='cuda:0')
tensor(0.3349, grad_fn=<SelectBackward0>)
inner tensor([[1, 0, 2, 1, 1, 2, 0, 0],
        [0, 1, 1, 2, 0, 0, 1, 2]], device='cuda:0')
    Processing edge type 2
tensor([[0, 1, 2],
        [0, 1, 2]], device='cuda:0')
tensor(0.3337, grad_fn=<SelectBackward0>)
inner tensor([[1, 0, 2, 1, 1, 2, 0, 0, 0, 1, 2],
        [0, 1, 1, 2, 0, 0, 1, 2, 0, 1, 2]], device='cuda:0')
after j tensor([[1, 0, 2, 1, 1, 2, 0, 0, 0, 1, 2],
        [0, 1, 1, 2, 0, 0, 1, 2, 0, 1, 2]], device='cuda:0')
tensor([0.3314, 0.3314, 0.3314, 0.3314, 0.3349, 0.3349, 0.3349, 0.3349, 0.3337,
        0.3337, 0.3337], device='cuda:0', grad_fn=<CatBackward0>)
index tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2],
        [0, 1, 2, 0, 1, 2, 0, 1, 2]], device='cuda:0')
valu tensor([0.3337, 0.6663

In [24]:
import torch
import numpy as np

# Dummy edges (replace with actual edges)
edges = [np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]]),  # Example edge 1
         np.array([[0, 1, 1], [1, 0, 0], [1, 0, 0]])]  # Example edge 2

num_nodes = 3  # Number of nodes (replace with actual number of nodes)

A = []
for i, edge in enumerate(edges):

    print(edge)
    #print(edge.nonzero()[1])
    edge_tmp = torch.from_numpy(np.vstack((edge.nonzero()[1], edge.nonzero()[0]))).type(torch.cuda.LongTensor)
    print(edge_tmp)
    value_tmp = torch.ones(edge_tmp.shape[1]).type(torch.cuda.FloatTensor)
    A.append((edge_tmp, value_tmp))

# Adding self-loop edge (assuming it's added for each node)
edge_tmp = torch.stack((torch.arange(0, num_nodes), torch.arange(0, num_nodes))).type(torch.cuda.LongTensor)
value_tmp = torch.ones(num_nodes).type(torch.cuda.FloatTensor)
A.append((edge_tmp, value_tmp))

# Print the dummy A
print("Dummy A:")
for i, (edge, value) in enumerate(A):
    print(f"Edge {i + 1}:")
    print("Edge tensor:")
    print(edge)
    print("Value tensor:")
    print(value)
    print()


[[0 1 0]
 [1 0 1]
 [0 1 0]]
tensor([[1, 0, 2, 1],
        [0, 1, 1, 2]], device='cuda:0')
[[0 1 1]
 [1 0 0]
 [1 0 0]]
tensor([[1, 2, 0, 0],
        [0, 0, 1, 2]], device='cuda:0')
Dummy A:
Edge 1:
Edge tensor:
tensor([[1, 0, 2, 1],
        [0, 1, 1, 2]], device='cuda:0')
Value tensor:
tensor([1., 1., 1., 1.], device='cuda:0')

Edge 2:
Edge tensor:
tensor([[1, 2, 0, 0],
        [0, 0, 1, 2]], device='cuda:0')
Value tensor:
tensor([1., 1., 1., 1.], device='cuda:0')

Edge 3:
Edge tensor:
tensor([[0, 1, 2],
        [0, 1, 2]], device='cuda:0')
Value tensor:
tensor([1., 1., 1.], device='cuda:0')



In [13]:
A

[(tensor([[1, 0, 2, 1],
          [0, 1, 1, 2]], device='cuda:0'),
  tensor([1., 1., 1., 1.], device='cuda:0')),
 (tensor([[1, 2, 0, 0],
          [0, 0, 1, 2]], device='cuda:0'),
  tensor([1., 1., 1., 1.], device='cuda:0')),
 (tensor([[0, 1, 2],
          [0, 1, 2]], device='cuda:0'),
  tensor([1., 1., 1.], device='cuda:0'))]

In [36]:
[(gtconv.weight),(gtconv.weight)]

[Parameter containing:
 tensor([[-0.0098,  0.0005, -0.0031],
         [-0.0005, -0.0025, -0.0085]], requires_grad=True),
 Parameter containing:
 tensor([[-0.0098,  0.0005, -0.0031],
         [-0.0005, -0.0025, -0.0085]], requires_grad=True)]

In [51]:
output_results[0]

(tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2],
         [0, 1, 2, 0, 1, 2, 0, 1, 2]], device='cuda:0'),
 tensor([0.3337, 0.6663, 0.3349, 0.6663, 0.3337, 0.3314, 0.3349, 0.3314, 0.3337],
        device='cuda:0', grad_fn=<CppNode<SegmentSumCSR>>))

In [38]:
a_edge, a_value = output_results[0]


tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2],
        [0, 1, 2, 0, 1, 2, 0, 1, 2]], device='cuda:0')

In [50]:
a=torch.sparse_coo_tensor(a_edge, a_value, (num_nodes, num_nodes)).to(a_edge.device)
a

tensor(indices=tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2],
                       [0, 1, 2, 0, 1, 2, 0, 1, 2]]),
       values=tensor([0.3337, 0.6663, 0.3349, 0.6663, 0.3337, 0.3314, 0.3349,
                      0.3314, 0.3337]),
       device='cuda:0', size=(5, 5), nnz=9, layout=torch.sparse_coo,
       grad_fn=<SparseCooTensorWithDimsAndTensorsBackward0>)

In [42]:
b=torch.sparse_coo_tensor(a_edge, a_value, (num_nodes, num_nodes)).to(a_edge.device)

In [65]:
 mat=torch.sparse.mm(a, b)
mat

tensor(indices=tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2],
                       [0, 1, 2, 0, 1, 2, 0, 1, 2]]),
       values=tensor([0.6675, 0.5557, 0.4443, 0.5557, 0.6652, 0.4443, 0.4443,
                      0.4443, 0.3333]),
       device='cuda:0', size=(5, 5), nnz=9, layout=torch.sparse_coo,
       grad_fn=<SparseSparseMatmulBackward0>)

In [56]:
mat.indices()

tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2],
        [0, 1, 2, 0, 1, 2, 0, 1, 2]], device='cuda:0')

In [49]:
torch.sparse.mm(output_results[0],output_results[1])

TypeError: _sparse_mm(): argument 'sparse' (position 1) must be Tensor, not tuple

In [61]:
row,col=mat.indices().detach()
row

tensor([0, 0, 0, 1, 1, 1, 2, 2, 2], device='cuda:0')

In [68]:
value=mat.values()

In [69]:
value.view(-1)

tensor([0.6675, 0.5557, 0.4443, 0.5557, 0.6652, 0.4443, 0.4443, 0.4443, 0.3333],
       device='cuda:0', grad_fn=<ViewBackward0>)

In [83]:
deg=scatter_add(value,row,dim=0,dim_size=3)
deg


tensor([1.6675, 1.6652, 1.2220], device='cuda:0',
       grad_fn=<ScatterAddBackward0>)

In [80]:
deg[row]*value

tensor([1.1130, 0.9266, 0.7409, 0.9253, 1.1076, 0.7399, 0.5430, 0.5430, 0.4073],
       device='cuda:0', grad_fn=<MulBackward0>)

In [84]:
output_results[0]

(tensor([[0, 0, 0, 1, 1, 1, 2, 2, 2],
         [0, 1, 2, 0, 1, 2, 0, 1, 2]], device='cuda:0'),
 tensor([0.3337, 0.6663, 0.3349, 0.6663, 0.3337, 0.3314, 0.3349, 0.3314, 0.3337],
        device='cuda:0', grad_fn=<CppNode<SegmentSumCSR>>))

In [85]:
 torch.full((num_nodes, ),
                        1)

tensor([1, 1, 1, 1, 1])

In [88]:
import torch

# Dummy data
num_nodes = 3
num_features = 10

# Randomly generated node features matrix (shape: 3 nodes x 10 features)
x = torch.randn(num_nodes, num_features)

# Edge weight vector (norm) of length 3 (assuming randomly generated weights)
norm = torch.tensor([0.8, 1.2, 0.9])  # Assuming edge weights for each node

# Edge index (assuming fully connected graph for simplicity)
edge_index = torch.tensor([[0, 1, 2],
                           [1, 2, 0]])  # Each column represents an edge (src, dst)

# Initialize aggregated messages matrix with zeros
aggregated_messages = torch.zeros_like(x)

# Iterate over each edge in the edge_index
for src, dst in edge_index.t().tolist():
    # Get the source node features
    src_features = x[src]
    print(src_features)
    print('norm',norm[src])
    # Compute the message to be passed from the source to the destination node
    message = src_features * norm[src]  # Element-wise multiplication with edge weight
    
    # Aggregate the message to the destination node
    aggregated_messages[dst] += message

print("Aggregated Messages Matrix:")
print(aggregated_messages)


tensor([ 0.2745, -0.3563,  0.4787, -0.1327,  1.1072,  0.2892,  0.0055,  0.4503,
         2.1566,  0.4243])
norm tensor(0.8000)
tensor([-0.0157, -0.7800, -0.0409, -0.5565,  0.2527,  1.1562, -0.2455,  0.0463,
        -0.1842, -0.4888])
norm tensor(1.2000)
tensor([-1.7192,  0.1621, -0.2766,  0.8944, -0.1968,  0.9488, -1.2888,  0.4265,
        -0.7933, -0.7006])
norm tensor(0.9000)
Aggregated Messages Matrix:
tensor([[-1.5473,  0.1459, -0.2489,  0.8050, -0.1771,  0.8540, -1.1599,  0.3838,
         -0.7139, -0.6305],
        [ 0.2196, -0.2850,  0.3829, -0.1061,  0.8857,  0.2314,  0.0044,  0.3602,
          1.7253,  0.3394],
        [-0.0188, -0.9360, -0.0491, -0.6678,  0.3032,  1.3874, -0.2946,  0.0555,
         -0.2211, -0.5865]])
